In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import librosa
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd
import pickle
import os
import h5py
import collections as cl
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
import random
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from deem import deem

In [2]:
param_grid = {'LR_param': {'C':[10**k for k in range(-10, 6, 2)]}, 'scoring': 'roc_auc', 'cv': 3, 'random_state': 42}

with open("openmic_classmap_10.json", "r") as f: # only consider 10 classes of Openmic dataset
    class_map = json.load(f)
    
# use a dict to align the classes between Openmic dataset (key) and Irmas dataset (val)
with open("class_align.json", "r") as f: # only consider 10 classes of Openmic dataset
    class_align = json.load(f)

with open("genre_map.json", "r") as f: # only consider 10 classes of Openmic dataset
    genre_map = json.load(f)
aligned_genre = list(genre_map)[:-1]

with open("instrument_map.json", "r") as f: # only consider 10 classes of Openmic dataset
    instrument_map = json.load(f)

## VGGish

In [3]:
embedding = 'vggish'
debias_method = '-k'

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid, class_align=class_align)

irmas_feature = deb.load_irmas()
openmic_feature = deb.load_openmic()

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

(13410, 128) (13410,)
(6705,)


100%|██████████| 6705/6705 [00:01<00:00, 4143.61it/s]


(6705, 128) (6705,)
(180000, 128) (180000,)


100%|██████████| 20000/20000 [01:21<00:00, 243.95it/s]


(20000, 128) (20000,)
(20000, 20) (20000, 20) (20000,)


...

100%|██████████| 10/10 [00:19<00:00,  1.92s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]


In [4]:
debias_method = '-klda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

Train on irmas, test on irmas


100%|██████████| 10/10 [00:19<00:00,  1.99s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [00:18<00:00,  1.85s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:19<00:00,  1.96s/it]


In [5]:
debias_method = '-mklda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = deb.result_all

Train on irmas, test on irmas


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [00:19<00:00,  1.95s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [00:20<00:00,  2.02s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [00:19<00:00,  1.99s/it]


## OpenL3

In [6]:
embedding = 'openl3'
debias_method = '-k'

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid, class_align=class_align)

irmas_feature = deb.load_irmas()
openmic_feature = deb.load_openmic()

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

(167625, 512) (167625,)
(6705,)


100%|██████████| 6705/6705 [00:30<00:00, 218.94it/s]


(6705, 512) (6705,)
(1900000, 512) (1900000,)


100%|██████████| 20000/20000 [15:52<00:00, 20.99it/s]


(20000, 512) (20000,)
(20000, 20) (20000, 20) (20000,)


...

100%|██████████| 10/10 [01:13<00:00,  7.39s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [01:13<00:00,  7.37s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [01:19<00:00,  7.92s/it]


In [7]:
debias_method = '-klda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

Train on irmas, test on irmas


100%|██████████| 10/10 [01:29<00:00,  8.95s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [01:10<00:00,  7.09s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [01:09<00:00,  6.96s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [01:15<00:00,  7.51s/it]


In [8]:
debias_method = '-mklda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = result_all.append(deb.result_all)

Train on irmas, test on irmas


100%|██████████| 10/10 [01:51<00:00, 11.19s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [01:17<00:00,  7.78s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [01:15<00:00,  7.59s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [01:18<00:00,  7.83s/it]


## YAMNet

In [9]:
embedding = 'yamnet'
debias_method = '-k'

deb = deem(embedding = embedding, debias_method = debias_method, feature_dir='./embeddings.h5', 
           instrument_map=instrument_map, genre_map=genre_map, param_grid=param_grid, class_align=class_align)

irmas_feature = deb.load_irmas()
openmic_feature = deb.load_openmic()

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

(33525, 1024) (33525,)
(6705,)


100%|██████████| 6705/6705 [00:04<00:00, 1610.25it/s]


(6705, 1024) (6705,)
(380000, 1024) (380000,)


100%|██████████| 20000/20000 [02:24<00:00, 138.38it/s]


(20000, 1024) (20000,)
(20000, 20) (20000, 20) (20000,)


...

100%|██████████| 10/10 [02:15<00:00, 13.52s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [02:12<00:00, 13.29s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [02:29<00:00, 14.92s/it]


In [10]:
debias_method = '-klda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

Train on irmas, test on irmas


100%|██████████| 10/10 [03:21<00:00, 20.20s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [03:31<00:00, 21.17s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [03:30<00:00, 21.09s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [03:26<00:00, 20.63s/it]


In [11]:
debias_method = '-mklda'

deb.debias_method = debias_method

deb.instrument_classfication(train_set='irmas', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='irmas', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='openmic', irmas_feature=irmas_feature, openmic_feature=openmic_feature)
deb.instrument_classfication(train_set='openmic', test_set='irmas', irmas_feature=irmas_feature, openmic_feature=openmic_feature)

deb.result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)
result_all = result_all.append(deb.result_all)

Train on irmas, test on irmas


100%|██████████| 10/10 [03:35<00:00, 21.60s/it]


Train on irmas, test on openmic


100%|██████████| 10/10 [03:39<00:00, 22.00s/it]


Train on openmic, test on openmic


100%|██████████| 10/10 [03:43<00:00, 22.35s/it]


Train on openmic, test on irmas


100%|██████████| 10/10 [03:39<00:00, 21.94s/it]


In [12]:
result_all.to_csv('results/result_nonlinear_all.csv', index=False)

In [13]:
result_all

,instrument,train_set,test_set,precision,recall,f1-score,support,accuracy,roc_auc,ap,embedding
0,cello,irmas,irmas,0.16417910447761194,0.9361702127659575,0.27936507936507937,94.0,0.7274909963985594,0.9276230306967679,0.5671656332279936,vggish-k
1,clarinet,irmas,irmas,0.24407582938388625,0.8110236220472441,0.37522768670309653,127.0,0.7941176470588235,0.8950335886376776,0.5736090213032011,vggish-k
2,flute,irmas,irmas,0.3286219081272085,0.768595041322314,0.4603960396039604,121.0,0.8691476590636255,0.8934338976704378,0.6012357681536926,vggish-k
3,guitar,irmas,irmas,0.5337954939341422,0.8901734104046243,0.66738894907909,346.0,0.8157262905162065,0.9225170782974251,0.7914319993453611,vggish-k
4,organ,irmas,irmas,0.47878787878787876,0.9080459770114943,0.626984126984127,174.0,0.8871548619447779,0.9639148254291084,0.7369438506943915,vggish-k
...,...,...,...,...,...,...,...,...,...,...,...
115,piano,openmic,irmas,0.1946222791293214,0.8941176470588236,0.3196635120925342,170.0,0.6116446578631453,0.8924307958477508,0.6936661598216468,yamnet-mklda
116,saxophone,openmic,irmas,0.22277227722772278,0.6,0.3249097472924188,150.0,0.7755102040816326,0.7689445910290237,0.36408564948623484,yamnet-mklda
117,trumpet,openmic,irmas,0.3188405797101449,0.88,0.4680851063829787,150.0,0.8199279711884754,0.9132717678100264,0.5555915194376949,yamnet-mklda
118,violin,openmic,irmas,0.21595330739299612,0.7762237762237763,0.3378995433789954,143.0,0.7388955582232893,0.8217678578807928,0.3894308689893178,yamnet-mklda
